In [2]:
%matplotlib qt
data_dir = input("Enter the data directory:")

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

Enter the data directory: H:\CNMFoutputs\jordan\tierN\20230819_PL43_wells101_jordan


In [3]:
%%time
## LOADING EVERYTHING UP - TAKES ~20 sec

# load packages

import napari
from magicgui import magicgui, widgets
import time

from IPython import get_ipython
from IPython.display import clear_output
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csc_matrix
from scipy import signal as sg
import scipy
import pickle

from tifffile.tifffile import imwrite,imread
from tqdm.auto import tqdm,trange

from copy import deepcopy
import h5py

import caiman as cm
from caiman.source_extraction.cnmf import cnmf,params
from caiman.paths import caiman_datadir
from caiman.utils.visualization import get_contours

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

def load_pickle(file_path):
    """
    Load a dictionary from a pickle file.

    Args:
    - file_path (str): Path to the pickle file.

    Returns:
    - dict: Loaded dictionary.
    """
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

## Loading all the inputs
os.chdir(data_dir)
#cnmf_path = caiman_datadir()+'/example_movies/demoMovie3DYxxbnobg_20240318170305_cnmf.hdf5'
cnmf_path = os.path.join(data_dir, 'ch0_means_movie_nobg_cnmf.hdf5')

# CNMFE model
cnmf_model = cnmf.load_CNMF(cnmf_path, 
                            n_processes=1,
                            dview=None)
print(f"Successfully loaded CNMF model")

mc_memmapped_fname = [i for i in os.listdir() if 'memmap__' in i][0]
Yr, dims, T = cm.load_memmap(mc_memmapped_fname)
images = np.array(np.reshape(Yr.T, [T] + list(dims), order='F')) 
print(f"Successfully loaded data")

#d = cnmf_model.estimates.A.shape[0]
#dims = cnmf_model.estimates.dims
#axis = 2
#order = list(range(4))
#order.insert(0, order.pop(axis))
#index_permut = np.reshape(np.arange(d), dims, order='F').transpose(
#        order[:-1]).reshape(d, order='F')
#A = csc_matrix(cnmf_model.estimates.A)[index_permut, :]
#dims = tuple(np.array(dims)[order[:3]])
#d1, d2, d3 = dims
#nr, T = cnmf_model.estimates.C.shape
#image_cells = np.array(A.mean(axis=1)).reshape(dims, order='F')
#coors = get_contours(A, dims, thr=Cthr)
coors = load_pickle(os.path.join(data_dir, 'ch0_means_movie_nobg_coors.pickle'))
print(f"Successfully loaded contours")

cc = [[l for l in n['coordinates']] for n in coors] # x,y values of contour coordinates for each component
cc1 = [[(l[:, 0]) for l in n['coordinates']] for n in coors] # x values of contour coordinates for each component
cc2 = [[(l[:, 1]) for l in n['coordinates']] for n in coors] # y values of contour coordinates for each component
length = np.ravel([list(map(len, cc)) for cc in cc1])
shapes = [[np.vstack([np.append(i,np.flip(pt)) for pt in cc[j][i]]) for i in range(len(cc[j]))] for j in range(len(cc))]

# Line up all static inputs
SNRs = cnmf_model.estimates.SNR_comp
SNR_min = cnmf_model.estimates.SNRmin
SOL = np.argsort(-SNRs)
spcomps = np.reshape(cnmf_model.estimates.A.toarray(),cnmf_model.estimates.dims + (-1,),order='F')
spcomps = spcomps.transpose([3,2,0,1])
images2 = images.transpose([0,3,1,2])
#SOL = np.argsort(-cnmf_model.estimates.SNR_comp) 
C = cnmf_model.estimates.C
CY = cnmf_model.estimates.C + cnmf_model.estimates.YrA # temporal loadings
R = cnmf_model.estimates.Craw # masks applied to raw movie

CYsav = cnmf_model.estimates.CYsav # smoothened CY curve
def sav_calc(sraw):
    return sg.savgol_filter(sraw,3,1)
Rsav = np.zeros(R.shape)
for i in range(R.shape[0]):
    Rsav[i,:] = sav_calc(R[i,:])
CYsavsort = np.sort(CYsav,axis=1)
CYsavb10 = np.mean(CYsavsort[:,:int(np.ceil(CYsavsort.shape[1]/10))],axis=1) # bottom 10% mean
Rsavsort = np.sort(Rsav,axis=1)
Rsavb10 = np.mean(Rsavsort[:,:int(np.ceil(Rsavsort.shape[1]/10))],axis=1) # bottom 10% mean
n_range = cnmf_model.estimates.n_range
if n_range is None:
    CYf = CYsavb10
    Rf = Rsavb10
else:
    CYf = np.mean(CY[:,n_range],axis=1)
    Rf = np.mean(R[:,n_range],axis=1)

Cn = cnmf_model.estimates.Cn # correlation image (not necessary)
keepargs = cnmf_model.estimates.keepargs
SOL = np.array([x for x in list(SOL) if x in list(keepargs)]) # initial search order list
print(f"Successfully generated search list")

A1 = csc_matrix(cnmf_model.estimates.A)
nr = A1.shape[1]
A_corr = scipy.sparse.triu(A1.T * A1)
A_corr.setdiag(0)
A_corr = A_corr.tocsc()
C_corr = scipy.sparse.lil_matrix(A_corr.shape)
for ii in range(nr):
    overlap_indices = scipy.sparse.find(A_corr[ii, :])[1][scipy.sparse.find(A_corr[ii, :])[2]>Athresh]
    if len(overlap_indices) > 0:
            # we chesk the correlation of the calcium traces for each overlapping components
        corr_values = [scipy.stats.pearsonr(C[ii, :], C[jj, :])[
            0] for jj in overlap_indices]
        C_corr[ii, overlap_indices] = corr_values
C_tot = C_corr + C_corr.T
CYR_corr = np.zeros(nr)
for ii in range(nr):
    CYR_corr[ii] = scipy.stats.pearsonr(R[ii,:],CY[ii,:])[0]
print(f"Successfully calculated correlations")

# Initialize all running variables in a single dictionary - just lists of arguments/component IDs (SOL, CSL, CMG, CKG, saved merge groups, trash)
# check if saved file exists - load that if it does, else instantiate new vars_dict!
save_path = 'ch0_means_movie_nobg_compfilt.pickle'
if save_path in os.listdir():
    vars_dict1 = load_pickle(save_path)
else:
    vars_dict1 = {
        "SOL": list(SOL), # search order list
        "CSL": list(), # current search list
        "CMG": list(), # current merge group
        "CKG": list(), # current keep group
        "SMG": list(), # saved merge groups (includes single components kept unmerged)
        "trash": list()
        #"trash": list(np.where(np.mean(CY[:,n_range],axis=1)<np.abs(np.min(np.mean(CY[:,n_range],axis=1))))[0]) # trash (all components to be removed) - start by removing all with too low baseline
    }
print(f"Successfully initialized")

Successfully loaded CNMF model
Successfully loaded data
Successfully loaded contours
Successfully generated search list
Successfully calculated correlations
Successfully initialized
CPU times: total: 29.9 s
Wall time: 37.6 s


In [4]:
## RUNNING THE GUI

# Click manager to route clicks into looper decisions
# state 0 = len(vars_dict["CSL"]) == 0
# need to make sure cand is a global variable available to all functions!
# remove cand from csl and sol in each function - need to be able to check state of each both before and after cand removed
def clickM(bypass=False):
    global vars_dict1, vars_dict2, vars_dict3
    if not bypass: # check if state = 0
        if len(vars_dict1["CSL"]) == 0:
            print_lab("Merge unavailable - first item in search group, try Keep or Trash")
            return
    vars_dict1["CSL"] = [x for x in vars_dict1["CSL"] if x not in [cand]]
    vars_dict1["SOL"] = [x for x in vars_dict1["SOL"] if x not in [cand]]
    vars_dict1["CMG"].append(cand)
    # every time an additional component is merged, reorder CMG by SNR and make top SNR component parent - actually no, keep first parent first!
    # vars_dict1["CMG"] = [vars_dict1["CMG"][i] for i in np.argsort(-SNRs[vars_dict1["CMG"]])]
    ovlp_cmp = [x for x in C_tot[cand,:].indices.tolist() if x in vars_dict1["SOL"]]
        # here need to remove anything previously processed from overlaps - e.g. keep only ovlp_cmp members in SOL (otherwise they're already in CMG, CSL, CKG, or trash)
    vars_dict1["SOL"] = [x for x in vars_dict1["SOL"] if x not in ovlp_cmp]
    vars_dict1["CSL"] = [x for n in (vars_dict1["CSL"],ovlp_cmp) for x in n]
    vars_dict1["CSL"] = [vars_dict1["CSL"][i] for i in np.argsort(-SNRs[vars_dict1["CSL"]])]
    looper()
    # add cand to CMG (merge shouldn't be an option from state 0 - in that case print an error and do nothing/wait for next click?)
    # add cand's children to CSL, remove cand and children from SOL, re-order CSL by SNR
    # run looper

def clickT():
    global vars_dict1, vars_dict2, vars_dict3
    vars_dict1["CSL"] = [x for x in vars_dict1["CSL"] if x not in [cand]]
    vars_dict1["SOL"] = [x for x in vars_dict1["SOL"] if x not in [cand]]
    vars_dict1["trash"].append(cand)
    looper()
    # add cand to trash list
    # run looper

def clickK():
    global vars_dict1, vars_dict2, vars_dict3
    if len(vars_dict1["CMG"]) == 0:
        clickM(bypass=True)
    else:
        vars_dict1["CSL"] = [x for x in vars_dict1["CSL"] if x not in [cand]]
        vars_dict1["SOL"] = [x for x in vars_dict1["SOL"] if x not in [cand]]
        vars_dict1["CKG"].append(cand)
        looper()
    # if from state 0, i.e. CMG empty - do as in clickM - call it with bypass=True
    # else if from state 1, i.e. CMG nonempty, add cand to CKG
    # run looper

def clickU(): #undo
    global vars_dict1, vars_dict2, vars_dict3
    # reverse vars_dict to one step back, then rerun looper
    vars_dict1 = deepcopy(vars_dict3)
    print_lab("Pressed undo")
    looper()

# Main looper function that updates data and viewer based on clicks
# Two states - depending on whether or not CSL is empty
def looper():
    global cand
    cand = -1
    
    global vars_dict1, vars_dict2, vars_dict3
    vars_dict3 = deepcopy(vars_dict2)
    vars_dict2 = deepcopy(vars_dict1)
        
    if len(vars_dict1["CSL"]) == 0: # merge is not an option in this state
        # start by adding CMG to SMG and clearing CMG
        # start by adding CKG (sorted by SNR) to front of SOL and clearing CKG
        if len(vars_dict1["CMG"]) > 0:
            vars_dict1["SMG"].append(vars_dict1["CMG"])
        vars_dict1["CMG"] = list()
        if len(vars_dict1["CKG"]) > 0:
            vars_dict1["SOL"] = vars_dict1["CKG"] + vars_dict1["SOL"]
        vars_dict1["CKG"] = list()

        if len(vars_dict1["SOL"]) == 0:
            parent_update()
            comp_update()
            prlab_update()
            
            clear_plot()
            print_lab("Congratulations - all done! Remember to press Save!")
            
        else:
            cand = vars_dict1["SOL"][0]
        
            parent_update()
            comp_update(comp=cand)
            prlab_update()

            clear_plot()
            parent_plot()
            comp_plot(comp=cand)
            plot_labels(comp=cand)

    else:
        cand = vars_dict1["CSL"][0]

        parent_update(paren=vars_dict1["CMG"])
        comp_update(comp=cand)
        prlab_update()

        clear_plot()
        parent_plot(paren=vars_dict1["CMG"],comp=cand)
        comp_plot(comp=cand)
        plot_labels(comp=cand)

# Initialize viewer and start GUI
viewer = napari.Viewer()
viewer.add_image(images.transpose([0,3,1,2]),name='cells',colormap='gray') 
#viewer.add_image(image_cells,name='cells',scale=[1,1,1]) # at some point may want to add images instead for scrolling - but fine for now

# initialize variables
#global vars_dict1, vars_dict2, vars_dict3
vars_dict2 = deepcopy(vars_dict1)
vars_dict3 = deepcopy(vars_dict1)

# Clicker GUI that lives in napari and runs functions (part of initialization)
@magicgui(
    O={
        "choices": ("Merge", "Trash", "Keep", "Undo"),
        "allow_multiple": True,
    }
)
def clicker(O=("Merge")):
    """Dropdown selection function."""
    print_lab()
    if 'Merge' in O:
        clickM()
    elif 'Trash' in O:
        clickT()
    elif 'Keep' in O:
        clickK()
    elif 'Undo' in O:
        clickU()

# Message displayer inside napari to print any messages
lab = widgets.Label()
def print_lab(message=None):
    if message is None:
        lab.value = ""
    else:
        lab.value = message

# Progress displayer inside napari
prlab = widgets.Label()
def prlab_update():
    prlab.value = str(len(vars_dict1['SOL']) + len(vars_dict1['CSL']) + len(vars_dict1['CKG'])) + " components to go"

# Save and close button inside napari
@magicgui(
    auto_call=True,btn={"widget_type": "PushButton", "text": "Save and close GUI"}
)
def save_btn(btn):
    with open(save_path, 'wb') as f:
        pickle.dump(vars_dict1, f)
    plt.close('all')
    viewer.close()

# Aligning widgets
layout = widgets.Container(
    widgets=[clicker,lab,save_btn,prlab], layout="vertical", labels=False
)

# Key bindings to speed up selections
@viewer.bind_key('u')
def pressU(viewer):
    clickU()

@viewer.bind_key('t')
def pressT(viewer):
    clickT()

@viewer.bind_key('m')
def pressM(viewer):
    clickM()

@viewer.bind_key('k')
def pressK(viewer):
    clickK()

@viewer.bind_key('Up')
def jump_up(viewer):
    viewer.dims.set_current_step(1, viewer.dims.current_step[1] - 1)

@viewer.bind_key('Down')
def jump_down(viewer):
    viewer.dims.set_current_step(1, viewer.dims.current_step[1] + 1)

@viewer.bind_key('Left')
def jump_left(viewer):
    viewer.dims.set_current_step(0, viewer.dims.current_step[0] - 1)

@viewer.bind_key('Right')
def jump_right(viewer):
    viewer.dims.set_current_step(0, viewer.dims.current_step[0] + 1)


# viewer updates
def comp_update(comp=None):
    try:
        viewer.layers.remove('component')
    except:
        pass
    try:
        viewer.layers.remove('component contours')
    except:
        pass
    if comp is not None:
        viewer.add_image(spcomps[comp,...],name='component',colormap='green',opacity=1,blending='additive',visible=False)
        viewer.add_points(np.vstack([g for g in [v[~np.isnan(v).any(axis=1)] for v in shapes[comp]] if g.size>0]),name='component contours',symbol='disc',size=2,face_color='lime',visible=True)
        viewer.camera.center = coors[comp].get('CoM')
        viewer.dims.set_point(1,coors[comp].get('CoM')[0])
        viewer.camera.zoom = 3 # seems to not run correctly the first time, confusing, bug?
        # update these functions to take candidate ID (or list of parent IDs) and render both the associated spcomps (filled volume - render invisible) and contours (outlines - render visible)
        # also could adjust contours to fit the scale factor - would need to include this upstream where cc/shapes arrays are being made - not worth it, need to interpolate etc...
        # also make it pan/zoom automatically to center of mass of candidate component in question
        
def parent_update(paren=None):
    try:
        viewer.layers.remove('parents')
    except:
        pass
    try:
        viewer.layers.remove('parent contours')
    except:
        pass
    if paren is not None:
        viewer.add_image(np.sum(spcomps[paren,...],axis=0),name='parents',colormap='darkorange',opacity=1,blending='additive',visible=True)
        viewer.add_points(np.vstack([g for g in [v[~np.isnan(v).any(axis=1)] for v in [j for k in [shapes[i] for i in paren] for j in k]] if g.size>0]),name='parent contours',symbol='disc',size=2,face_color='orange',visible=False)

# Plotting functions - the idea here is that there are two figures that update every turn (both should clear once per round - maybe implement clearing part in looper/as separate function)
# One figure is dF/F, the other is fluorescence - both have two axes - one for candidate(s), one for parent(s)
# In particular, component should plot: dF/F - candidate component in bright, remaining search list in dim, candidate (raw) dotted; fluorescence - candidate component timecourse
# In particular, parent should plot: dF/F - main parent in bright, remaining merge group in dim, mean merge group (raw) dotted; fluorescence - main parent component timecourse
# Key purposes of these plots:
    # 1. Comparing curve shapes - should make sense, parent/merge group should match, component/raw should match
    # 2. Comparing magnitudes - components with very low/negative magnitudes, or mags much smaller than parent, don't make sense
# If a plotting function gets an empty argument - just don't plot anything

global fig1, fig2, ax11, ax21, ax12, ax22, leg1, leg2
fig1, ax11 = plt.subplots() # dF/F
ax12 = ax11.twinx()
fig2, ax21 = plt.subplots() # fluorescence
ax22 = ax21.twinx()
fig1.suptitle('dF/F plots')
fig2.suptitle('Fluorescence signal plots')
ax11.plot(CY[0,:].T/100,c='green',label='initialization')
ax21.plot(CY[0,:].T/100,c='green',label='initialization')
leg1 = fig1.legend(loc="upper left")
leg2 = fig2.legend(loc="upper left")

# On fig1/ax11: should plot candidate bright and search list dim, candidate raw dotted
# On fig2/ax21: should plot candidate timecourse C+YrA
# Make sure titles are clear - e.g. dF/F and fluorescence curves
# Also add legend with key parameters of candidate - SNR and correlation with main parent timecourse
# Maybe then just also add a legent with candidate in green/parent in orange
def comp_plot(comp=None):
    if comp is not None:
        ax11.plot((CY[comp,:].T-CYf[comp])/CYf[comp],c='green',label='component')  # very important to plot the right stuff here in terms of y-axis, dynamic range, etc for optimal clicking
        ax11.plot((R[comp,:].T-Rf[comp])/Rf[comp],c='green',ls='--',label='component raw')
        if len(vars_dict1["CSL"]) > 0:
            ax11.plot((CY[vars_dict1["CSL"],:].T-CYf[np.array(vars_dict1["CSL"])])/CYf[np.array(vars_dict1["CSL"])],c='green',alpha=0.2)
        fig1.canvas.draw_idle()
        #plt.plot((C[0,:].T-np.mean(C[0,n_range]))/np.mean(C[0,n_range]))  # very important to plot the right stuff here in terms of y-axis, dynamic range, etc for optimal clicking
        #plt.figure(2)
        ax21.plot(CY[comp,:].T/100,c='green',label='component')
        fig2.canvas.draw_idle()

# On fig1/ax12: should plot main parent bright and merge list dim, parent raw (mean) dotted
# On fig2/ax22: should plot main parent timecourse C+YrA
def parent_plot(paren=None,comp=None):
    if paren is not None:
        ax12.plot((CY[paren[0],:].T-CYf[paren[0]])/CYf[paren[0]],c='darkorange',label='first parent')
        ax12.plot(np.mean((R[paren,:].T-Rf[np.array(paren)])/Rf[np.array(paren)],axis=1),c='darkorange',ls='--',label='parents raw')
        ax12.plot(np.mean((R[paren+[comp],:].T-Rf[np.array(paren+[comp])])/Rf[np.array(paren+[comp])],axis=1),c='gold',ls='--',label='merged raw')
        #ax12.plot((CY[paren,:].T-np.mean(CY[np.array(paren)[:,None],n_range[None,:]],axis=1))/np.mean(CY[np.array(paren)[:,None],n_range[None,:]],axis=1))
        ax12.plot((CY[paren,:].T-CYf[np.array(paren)])/CYf[np.array(paren)],c='darkorange',alpha=0.2)
        fig1.canvas.draw_idle()
        # Rethink these - either order CMG differently (not by SNR but by e.g. peak dF/F?) or just compare component to parent raw mean? Or both?
        ax22.plot(np.mean(R[paren,:],axis=0).T,c='darkorange',ls='--',label='parents raw')
        #ax22.plot(CY[paren[0],:].T/100,c='darkorange',label='parent')
        y_lim = ax22.get_ylim()
        ax22.plot(np.mean(R[paren+[comp],:],axis=0).T,c='gold',ls='--',label='merged raw')
        #ax22.plot(CY[paren,:].T/100,c='darkorange',alpha=0.2,label='parents')
        ax22.set_ylim(y_lim)
        fig2.canvas.draw_idle()

def plot_labels(comp):
    global leg1,leg2
    leg1.remove()
    leg2.remove()
    leg1 = fig1.legend(loc="upper left")
    leg2 = fig2.legend(loc="upper left")
    align_yaxis(ax11,ax12)
    # Here make text boxes with key info: 
    # Anything about SNR? Probably TMI
    # Temporal correlations: candidate to first and to best parent, candidate to its raw trace
    # Helper messages: likely merge if candidate to best parent correlation > 0.9/0.95?, possible trash if candidate to its raw correlation < 0.9/0.85?
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.2)
    props2 = dict(boxstyle='square', facecolor='red', alpha=0.2)
    props3 = dict(boxstyle='square', facecolor='green', alpha=0.2)
    cr_corr = CYR_corr[comp]
    textstr1 = '\n'.join((
        r'comp-raw corr=$%.2f$' % (cr_corr, ),
        ))
    ax11.text(0.05, 0.8, textstr1, transform=ax11.transAxes, fontsize=12,
        verticalalignment='top', bbox=props)
    if cr_corr < cr_thresh:
        ax11.text(0.05, 0.7, 'possible trash',fontsize=12,
            verticalalignment='top', transform=ax11.transAxes, bbox=props2)
    if len(vars_dict1["CMG"]) > 0:
        pf_corr = C_tot[comp,vars_dict1["CMG"][0]]
        pb_corr = np.max(C_tot[comp,vars_dict1["CMG"]])
        textstr2 = '\n'.join((
            r'best parent corr=$%.2f$' % (pb_corr, ),
            r'first parent corr=$%.2f$' % (pf_corr, ),
            ))
        ax21.text(0.05, 0.9, textstr2, transform=ax21.transAxes, fontsize=12,
            verticalalignment='top', bbox=props)
        if pb_corr > pb_thresh:
            ax21.text(0.05, 0.7, 'likely merge',fontsize=12,
                verticalalignment='top', transform=ax21.transAxes, bbox=props3)
    
def clear_plot():
    plt.sca(ax11)
    plt.cla()
    plt.sca(ax12)
    plt.cla()
    plt.sca(ax21)
    plt.cla()
    plt.sca(ax22)
    plt.cla()
    ax11.set_xlabel('Frame')
    ax11.set_ylabel('dF/F')
    #ax12.set_ylabel('dF/F (parent)', color='orange')
    ax21.set_xlabel('Frame')
    ax21.set_ylabel('Signal')
    #ax22.set_ylabel('Signal (parent)', color='orange')
    ax11.tick_params(axis='y', labelcolor='green')
    ax21.tick_params(axis='y', labelcolor='green')
    ax12.tick_params(axis='y', labelcolor='darkorange')
    ax22.tick_params(axis='y', labelcolor='darkorange')

def align_yaxis(ax1, ax2):
    y_lims = np.array([ax.get_ylim() for ax in [ax1, ax2]])

    # force 0 to appear on both axes, comment if don't need
    y_lims[:, 0] = y_lims[:, 0].clip(None, 0)
    y_lims[:, 1] = y_lims[:, 1].clip(0, None)

    # normalize both axes
    y_mags = (y_lims[:,1] - y_lims[:,0]).reshape(len(y_lims),1)
    y_lims_normalized = y_lims / y_mags

    # find combined range
    y_new_lims_normalized = np.array([np.min(y_lims_normalized), np.max(y_lims_normalized)])

    # denormalize combined range to get new axes
    new_lim1, new_lim2 = y_new_lims_normalized * y_mags
    ax1.set_ylim(new_lim1)
    ax2.set_ylim(new_lim2)

#my_widget.show(run=True)

#viewer = napari.view_image(image_neurons)
viewer.window.add_dock_widget(layout)
looper()